In [1]:
import os
import pathlib
import torch

import numpy as np
import pandas as pd
import plotly.graph_objects as go
from plotly.subplots import make_subplots
from plotly_resampler import unregister_plotly_resampler

from neuralprophet import NeuralProphet, set_random_seed

In [2]:
def create_metrics_plot(metrics):
    # Deactivate the resampler since it is not compatible with kaleido (image export)
    unregister_plotly_resampler()

    # Plotly params
    prediction_color = "#2d92ff"
    actual_color = "black"
    line_width = 2
    xaxis_args = {"showline": True, "mirror": True, "linewidth": 1.5, "showgrid": False}
    yaxis_args = {
        "showline": True,
        "mirror": True,
        "linewidth": 1.5,
        "showgrid": False,
        "rangemode": "tozero",
        "type": "log",
    }
    layout_args = {
        "autosize": True,
        "template": "plotly_white",
        "margin": go.layout.Margin(l=0, r=10, b=0, t=30, pad=0),
        "font": dict(size=10),
        "title": dict(font=dict(size=10)),
        "width": 1000,
        "height": 200,
    }

    metric_cols = [col for col in metrics.columns if not ("_val" in col or col == "RegLoss" or col == "epoch")]
    fig = make_subplots(rows=1, cols=len(metric_cols), subplot_titles=metric_cols)
    for i, metric in enumerate(metric_cols):
        fig.add_trace(
            go.Scatter(
                y=metrics[metric],
                name=metric,
                mode="lines",
                line=dict(color=prediction_color, width=line_width),
                legendgroup=metric,
            ),
            row=1,
            col=i + 1,
        )
        if f"{metric}_val" in metrics.columns:
            fig.add_trace(
                go.Scatter(
                    y=metrics[f"{metric}_val"],
                    name=f"{metric}_val",
                    mode="lines",
                    line=dict(color=actual_color, width=line_width),
                    legendgroup=metric,
                ),
                row=1,
                col=i + 1,
            )
        if metric == "Loss":
            fig.add_trace(
                go.Scatter(
                    y=metrics["RegLoss"],
                    name="RegLoss",
                    mode="lines",
                    line=dict(color=actual_color, width=line_width),
                    legendgroup=metric,
                ),
                row=1,
                col=i + 1,
            )
    fig.update_xaxes(xaxis_args)
    fig.update_yaxes(yaxis_args)
    fig.update_layout(layout_args)
    return fig

In [3]:
DIR = "~/github/neural_prophet"
DATA_DIR = os.path.join(DIR, "tests", "test-data")
PEYTON_FILE = os.path.join(DATA_DIR, "wp_log_peyton_manning.csv")
AIR_FILE = os.path.join(DATA_DIR, "air_passengers.csv")
YOS_FILE = os.path.join(DATA_DIR, "yosemite_temps.csv")
ENERGY_PRICE_DAILY_FILE = os.path.join(DATA_DIR, "tutorial04_kaggle_energy_daily_temperature.csv")

In [4]:
df = pd.read_csv(ENERGY_PRICE_DAILY_FILE)
df["temp"] = df["temperature"]
df = df.drop(columns="temperature")
df["ds"] = pd.to_datetime(df["ds"])
df["y"] = pd.to_numeric(df["y"], errors="coerce")

df = df.drop("ds", axis=1)
df["ds"] = pd.date_range(start="2015-01-01 00:00:00", periods=len(df), freq="h")
df["ID"] = "test"

df_id = df[["ds", "y", "temp"]].copy()
df_id["ID"] = "test2"
df_id["y"] = df_id["y"] * 0.3
df_id["temp"] = df_id["temp"] * 0.4
df = pd.concat([df, df_id], ignore_index=True)

# Conditional Seasonality
df["winter"] = np.where(
    df["ds"].dt.month.isin([1]),
    1,
    0,
)
df["summer"] = np.where(df["ds"].dt.month.isin([2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12]), 1, 0)
df["winter"] = pd.to_numeric(df["winter"], errors="coerce")
df["summer"] = pd.to_numeric(df["summer"], errors="coerce")

# Normalize Temperature
df["temp"] = (df["temp"] - 65.0) / 50.0

# df
df = df[["ID", "ds", "y", "temp", "winter", "summer"]]

# Split
df_train = df[df["ds"] < "2015-03-01"]
df_test = df[df["ds"] >= "2015-03-01"]

In [5]:
# Hyperparameter
tuned_params = {
    "n_lags": 10,
    "newer_samples_weight": 2.0,
    "n_changepoints": 0,
    "yearly_seasonality": 10,
    "weekly_seasonality": True,
    "daily_seasonality": False,  # due to conditional daily seasonality
    "batch_size": 64,
    "ar_layers": [8, 4],
    "lagged_reg_layers": [8],
    # not tuned
    "n_forecasts": 5,
    "learning_rate": 0.1,
    "epochs": 20,
    "trend_global_local": "global",
    "season_global_local": "global",
    "drop_missing": True,
    "normalize": "standardize",
}

# Uncertainty Quantification
confidence_lv = 0.98
quantile_list = [round(((1 - confidence_lv) / 2), 2), round((confidence_lv + (1 - confidence_lv) / 2), 2)]
# quantile_list = None
print(f"quantiles: {quantile_list}")

# Check if GPU is available
# use_gpu = torch.cuda.is_available()
use_gpu = False

# Set trainer configuration
trainer_configs = {
    "accelerator": "gpu" if use_gpu else "cpu",
}
print(f"Using {'GPU' if use_gpu else 'CPU'}")

# Model
m = NeuralProphet(**tuned_params, **trainer_configs, quantiles=quantile_list)

# Lagged Regressor
m.add_lagged_regressor(names="temp", n_lags=33, normalize="standardize")

# Conditional Seasonality
m.add_seasonality(name="winter", period=1, fourier_order=6, condition_name="winter")
m.add_seasonality(name="summer", period=1, fourier_order=6, condition_name="summer")

# Holidays
m.add_country_holidays(country_name="US", lower_window=-1, upper_window=1)

quantiles: [0.01, 0.99]
Using CPU


In [6]:
# Training & Predict
metrics = m.fit(
    df=df_train,
    validation_df=df_test,
    freq="h",
    early_stopping=False,
    scheduler="onecyclelr",
    scheduler_args={
        "pct_start": 0.3,
        "div_factor": 100.0,
        "final_div_factor": 1000.0,
        "anneal_strategy": "cos",
        "three_phase": False,
    },
    # scheduler="exponentiallr",
    # scheduler_args={"gamma": 0.8,},
)

INFO - (NP.forecaster.fit) - When Global modeling with local normalization, metrics are displayed in normalized scale.
WARNING - (py.warnings._showwarnmsg) - /home/tabletop/github/neural_prophet/neuralprophet/df_utils.py:1149: FutureWarning: Series.view is deprecated and will be removed in a future version. Use ``astype`` as an alternative to change the dtype.
  converted_ds = pd.to_datetime(ds_col, utc=True).view(dtype=np.int64)

INFO - (NP.df_utils._infer_frequency) - Major frequency h corresponds to 99.929% of the data.
WARNING - (py.warnings._showwarnmsg) - /home/tabletop/github/neural_prophet/neuralprophet/df_utils.py:1149: FutureWarning: Series.view is deprecated and will be removed in a future version. Use ``astype`` as an alternative to change the dtype.
  converted_ds = pd.to_datetime(ds_col, utc=True).view(dtype=np.int64)

WARNING - (py.warnings._showwarnmsg) - /home/tabletop/github/neural_prophet/neuralprophet/df_utils.py:1149: FutureWarning: Series.view is deprecated and wi

Training: |          | 0/? [00:00<?, ?it/s]

WARNING - (py.warnings._showwarnmsg) - /home/tabletop/github/neural_prophet/neuralprophet/time_dataset.py:692: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  contains_nan = torch.cat([torch.tensor(contains_nan), torch.ones(n_forecasts, dtype=torch.bool)])

WARNING - (py.warnings._showwarnmsg) - /home/tabletop/github/neural_prophet/neuralprophet/time_dataset.py:692: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  contains_nan = torch.cat([torch.tensor(contains_nan), torch.ones(n_forecasts, dtype=torch.bool)])



Training: |          | 0/? [00:00<?, ?it/s]

WARNING - (py.warnings._showwarnmsg) - /home/tabletop/.cache/pypoetry/virtualenvs/neuralprophet-CT7lk1Bv-py3.10/lib/python3.10/site-packages/torch/optim/lr_scheduler.py:232: UserWarning: The epoch parameter in `scheduler.step()` was not necessary and is being deprecated where possible. Please use `scheduler.step()` to step the scheduler. During the deprecation, if epoch is different from None, the closed form is used instead of the new chainable form, where available. Please open an issue if you are unable to replicate your use case: https://github.com/pytorch/pytorch/issues/new/choose.
  warnings.warn(EPOCH_DEPRECATION_WARNING, UserWarning)



...

Validation: |          | 0/? [00:00<?, ?it/s]

Validation: |          | 0/? [00:00<?, ?it/s]

Validation: |          | 0/? [00:00<?, ?it/s]

Validation: |          | 0/? [00:00<?, ?it/s]

Validation: |          | 0/? [00:00<?, ?it/s]

In [7]:
create_metrics_plot(metrics)

In [8]:
metrics.to_dict("records")[-1]

{'MAE_val': 0.5085433125495911,
 'RMSE_val': 0.5813770294189453,
 'Loss_val': 0.3913739025592804,
 'RegLoss_val': 0.0,
 'epoch': 19,
 'MAE': 0.4592474699020386,
 'RMSE': 0.6220942139625549,
 'Loss': 0.26762738823890686,
 'RegLoss': 0.0,
 'LR': 0.0004054287273902446}

In [9]:
metrics

,MAE_val,RMSE_val,Loss_val,RegLoss_val,epoch,MAE,RMSE,Loss,RegLoss,LR
0,0.499936,0.583346,0.938270,0.0,0,1.503294,2.114124,1.916612,0.0,0.004087
1,0.534045,0.631530,0.440998,0.0,1,0.718145,0.943761,0.505523,0.0,0.021600
2,0.542755,0.644081,0.454675,0.0,2,0.537536,0.724863,0.347341,0.0,0.050152
3,0.508438,0.616892,0.487512,0.0,3,0.503906,0.677358,0.312197,0.0,0.078837
4,0.649246,0.755430,0.545550,0.0,4,0.505661,0.671692,0.313073,0.0,0.096699
5,0.463848,0.568442,0.367994,0.0,5,0.520102,0.691615,0.322044,0.0,0.099596
6,0.355072,0.410634,0.251356,0.0,6,0.511964,0.684423,0.316359,0.0,0.097137
7,0.447367,0.500184,0.336913,0.0,7,0.503181,0.669457,0.307173,0.0,0.092315
8,0.821846,0.951728,0.720978,0.0,8,0.503031,0.671102,0.308114,0.0,0.085371
9,0.414638,0.474769,0.334302,0.0,9,0.511291,0.686945,0.311271,0.0,0.076654


In [10]:
forecast = m.predict(df)

WARNING - (py.warnings._showwarnmsg) - /home/tabletop/github/neural_prophet/neuralprophet/df_utils.py:1149: FutureWarning:

Series.view is deprecated and will be removed in a future version. Use ``astype`` as an alternative to change the dtype.


INFO - (NP.df_utils._infer_frequency) - Major frequency h corresponds to 99.932% of the data.
WARNING - (py.warnings._showwarnmsg) - /home/tabletop/github/neural_prophet/neuralprophet/df_utils.py:1149: FutureWarning:

Series.view is deprecated and will be removed in a future version. Use ``astype`` as an alternative to change the dtype.


WARNING - (py.warnings._showwarnmsg) - /home/tabletop/github/neural_prophet/neuralprophet/df_utils.py:1149: FutureWarning:

Series.view is deprecated and will be removed in a future version. Use ``astype`` as an alternative to change the dtype.


INFO - (NP.df_utils._infer_frequency) - Defined frequency is equal to major frequency - h
WARNING - (py.warnings._showwarnmsg) - /home/tabletop/github/neural_prophet

Predicting: |          | 0/? [00:00<?, ?it/s]

WARNING - (py.warnings._showwarnmsg) - /home/tabletop/github/neural_prophet/neuralprophet/time_dataset.py:692: UserWarning:

To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).




Predicting: |          | 0/? [00:00<?, ?it/s]

In [11]:
m.highlight_nth_step_ahead_of_each_forecast(m.n_forecasts)
m.plot(forecast, df_name="test")

INFO - (NP.forecaster.plot) - Plotting data from ID test
WARNING - (py.warnings._showwarnmsg) - /home/tabletop/github/neural_prophet/neuralprophet/plot_forecast_plotly.py:100: FutureWarning:

The behavior of DatetimeProperties.to_pydatetime is deprecated, in a future version this will return a Series containing python datetime objects instead of an ndarray. To retain the old behavior, call `np.array` on the result


WARNING - (py.warnings._showwarnmsg) - /home/tabletop/.cache/pypoetry/virtualenvs/neuralprophet-CT7lk1Bv-py3.10/lib/python3.10/site-packages/plotly_resampler/figure_resampler/utils.py:177: FutureWarning:

'H' is deprecated and will be removed in a future version. Please use 'h' instead of 'H'.


WARNING - (py.warnings._showwarnmsg) - /home/tabletop/.cache/pypoetry/virtualenvs/neuralprophet-CT7lk1Bv-py3.10/lib/python3.10/site-packages/plotly_resampler/figure_resampler/utils.py:178: FutureWarning:

'H' is deprecated and will be removed in a future version, please use 'h' inst

FigureWidgetResampler({
    'data': [{'fillcolor': 'rgba(45, 146, 255, 0.2)',
              'line': {'color': 'rgba(45, 146, 255, 0.2)', 'width': 1},
              'mode': 'lines',
              'name': '<b style="color:sandybrown">[R]</b> yhat5 1.0% <i style="color:#fc9944">~1h</i>',
              'type': 'scatter',
              'uid': '1e41dda2-f7ca-4501-ae0d-394dbc69313f',
              'x': array([datetime.datetime(2015, 1, 2, 13, 0),
                          datetime.datetime(2015, 1, 2, 14, 0),
                          datetime.datetime(2015, 1, 2, 15, 0), ...,
                          datetime.datetime(2015, 3, 2, 17, 0),
                          datetime.datetime(2015, 3, 2, 18, 0),
                          datetime.datetime(2015, 3, 2, 20, 0)], dtype=object),
              'y': array([ 7.0392876,  9.7315445, 17.582043 , ..., 48.029076 , 46.43782  ,
                          48.867878 ], dtype=float32)},
             {'fill': 'tonexty',
              'fillcolor': 'rgba(45

In [12]:
m.plot_components(forecast, df_name="test")

INFO - (NP.forecaster.plot_components) - Plotting data from ID test
WARNING - (py.warnings._showwarnmsg) - /home/tabletop/github/neural_prophet/neuralprophet/plot_forecast_plotly.py:410: FutureWarning:

The behavior of DatetimeProperties.to_pydatetime is deprecated, in a future version this will return a Series containing python datetime objects instead of an ndarray. To retain the old behavior, call `np.array` on the result


WARNING - (py.warnings._showwarnmsg) - /home/tabletop/.cache/pypoetry/virtualenvs/neuralprophet-CT7lk1Bv-py3.10/lib/python3.10/site-packages/plotly_resampler/figure_resampler/utils.py:177: FutureWarning:

'H' is deprecated and will be removed in a future version. Please use 'h' instead of 'H'.


WARNING - (py.warnings._showwarnmsg) - /home/tabletop/.cache/pypoetry/virtualenvs/neuralprophet-CT7lk1Bv-py3.10/lib/python3.10/site-packages/plotly_resampler/figure_resampler/utils.py:178: FutureWarning:

'H' is deprecated and will be removed in a future version, please u

IndexError: index -1 is out of bounds for axis 0 with size 0

In [ ]:
m.plot_parameters()

WARNING - (py.warnings._showwarnmsg) - /home/tabletop/github/neural_prophet/neuralprophet/plot_model_parameters_plotly.py:178: FutureWarning:

The behavior of DatetimeProperties.to_pydatetime is deprecated, in a future version this will return a Series containing python datetime objects instead of an ndarray. To retain the old behavior, call `np.array` on the result


WARNING - (py.warnings._showwarnmsg) - /home/tabletop/github/neural_prophet/neuralprophet/plot_model_parameters_plotly.py:475: FutureWarning:

The behavior of DatetimeProperties.to_pydatetime is deprecated, in a future version this will return a Series containing python datetime objects instead of an ndarray. To retain the old behavior, call `np.array` on the result


WARNING - (py.warnings._showwarnmsg) - /home/tabletop/github/neural_prophet/neuralprophet/plot_model_parameters_plotly.py:508: FutureWarning:

The behavior of DatetimeProperties.to_pydatetime is deprecated, in a future version this will return a Series conta

FigureWidgetResampler({
    'data': [{'fill': 'none',
              'line': {'color': '#2d92ff', 'width': 2},
              'mode': 'lines',
              'name': 'Trend',
              'type': 'scatter',
              'uid': '2b2cb1c3-4a0e-4729-92d7-1167a376d732',
              'x': array([datetime.datetime(2015, 1, 1, 0, 0),
                          datetime.datetime(2015, 2, 28, 23, 0)], dtype=object),
              'xaxis': 'x',
              'y': array([25.74136 , 14.885769], dtype=float32),
              'yaxis': 'y'},
             {'fill': 'none',
              'line': {'color': '#2d92ff', 'width': 2},
              'mode': 'lines',
              'name': 'yearly',
              'type': 'scatter',
              'uid': '965409ff-c889-48d6-bae0-936574b46f88',
              'x': array([datetime.datetime(2017, 1, 1, 0, 0),
                          datetime.datetime(2017, 1, 2, 0, 0),
                          datetime.datetime(2017, 1, 3, 0, 0), ...,
                          datet